In [66]:
!pip install InstructorEmbedding

In [67]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,RetrievalQAWithSourcesChain
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
import langchain
from getpass import getpass

In [52]:
HUGGINGFACEHUB_API_TOKEN = getpass()

········


In [53]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [54]:
loader=UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/world/icc-world-cup-2023-england-win-toss-bat-and-pakistan-all-but-out-of-tournament-11724881.html",
    "https://www.moneycontrol.com/news/cricket/icc-world-cup-2023-wc-hits-1-mn-fans-milestone-on-track-to-become-one-of-the-most-attended-icc-events-11724541.html"
])
data=loader.load()

In [55]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)


In [56]:
docs=text_splitter.split_documents(data)
len(docs)

38

In [40]:
docs[4]

Document(page_content='.header_desktop.header1024 .investNW .sepInvsli a.pdg_btm{padding-bottom: 0px;}\n\t\t@media screen and (max-width:1279px) { \n\t\t\t.investNW .sepInvsli a.pdg_btm{ padding-bottom: 0px; }', metadata={'source': 'https://www.moneycontrol.com/news/world/icc-world-cup-2023-england-win-toss-bat-and-pakistan-all-but-out-of-tournament-11724881.html'})

In [41]:
docs[0].page_content

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nTrending Stocks\n\nSuzlon Energy\xa0INE040H01021, SUZLON, 532667\n\nESAF SFB\xa0INE818W01011, ESAFSFB, 544020\n\nTata Power\xa0INE245A01021, TATAPOWER, 500400\n\nJaiprakash Pow\xa0INE351F01018, JPPOWER, 532627\n\nYes Bank\xa0INE528G01035, YESBANK, 532648\n\nCheck your Credit Score here!\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, LoginHello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistMy Credit ScoreMy FeedMy MessagesMy AlertsMy Profile My PROMy PortfolioMy WatchlistMy Credit ScoreMy FeedMy MessagesMy AlertsLogoutChat with UsDownload AppFollow us on:\n\nUpgrade\n\nMy Feed'

In [68]:
model_name="hkunlp/instructor-large"
hf=HuggingFaceInstructEmbeddings(model_name=model_name)

vector_index=FAISS.from_documents(docs,hf)

    


load INSTRUCTOR_Transformer
max_seq_length  512


In [24]:
vector.shape

(768,)

In [69]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", model_kwargs={"temperature":0.9, "max_length": 500}
)

C:\Users\darre\anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [70]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vector_index.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=HuggingFaceHub(client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text-generation/HuggingFaceH4/zephyr-7b-beta', task='text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={'temperature': 0.9, 'max_length': 500})), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you d

In [71]:
query='when did the millionth fan came?'

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "when did the millionth fan came?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "With six games to go, the millionth fan came through the turnstiles during the match between Afghanistan and South Africa at the Narendra Modi Stadium in Ahmedabad on November 10.\n\nOver a million fans have attended the ongoing cricket World Cup with the tournament on track to become one of the most attended ICC events in history. With six games to go, the millionth fan came through the turnstiles during the match between Afghanistan and South Africa at the Narendra Modi Stadium in Ahmedabad on November 10, the ICC said in a media release.\n\nThe tour

[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain > 5:llm:HuggingFaceHub] [3.15s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " \"With six games to go, the millionth fan came through the turnstiles during the match",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain > 6:llm:HuggingFaceHub] [3.15s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " \"With over 1 million attendees and record-breaking viewership, the ICC Men",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain > 7:llm:HuggingFaceHub] [3.15s] Exiting LLM run with output:
{
  "g

[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 9:chain:LLMChain > 10:llm:HuggingFaceHub] [3.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " No information is provided about when the millionth fan came.\nSOURCES:\n\n",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 9:chain:LLMChain] [3.24s] Exiting Chain run with output:
{
  "text": " No information is provided about when the millionth fan came.\nSOURCES:\n\n"
}
[chain/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] [6.97s] Exiting Chain run with output:
{
  "output_text": " No information is provided about when the millionth fan came.\nSOURCES:\n\n"
}
[chain/end] [1:chain:RetrievalQAWithSourcesChain] [8.11s] Exiting Chain run with output:
{
  "answer": " No information is provide

{'answer': ' No information is provided about when the millionth fan came.\n',
 'sources': ''}